In [ ]:
import jax
import jax.numpy as jnp
import numpy as np

from kinetix.transport import (
    Advection,
    Cells,
    Dispersion,
    FixedConcentrationBoundary,
    Species,
    System,
    make_solver,
)

In [ ]:
jax.config.update("jax_enable_x64", True)

In [ ]:
cells = Cells.equally_spaced(10, 200)

In [ ]:
dispersion = Dispersion(
    dispersivity=jnp.array(0.1),
    pore_diffusion=Species(
        tracer=jnp.array(1e-9 * 3600 * 24),
    ),
)

In [ ]:
advection = Advection(
    limiter_type="minmod"
)

In [ ]:
bcs = [
    FixedConcentrationBoundary(
        is_active=lambda t, system: True,
        left=True,
        species_selector=lambda s: getattr(s, "tracer"),
        fixed_concentration=lambda t: jnp.array(10.0),
    ),
    FixedConcentrationBoundary(
        is_active=lambda t, system: True,
        left=False,
        species_selector=lambda s: getattr(s, "tracer"),
        fixed_concentration=lambda t: jnp.array(3.0),
    )
]

In [ ]:
system = System(
    porosity=jnp.array(0.3),
    velocity=lambda t: jnp.array(1 / 365) * jnp.sin(np.pi * 2 * 1 / 5000 * t),
    cells=cells,
    advection=advection,
    dispersion=dispersion,
    bcs=bcs
)

In [ ]:
t_points = jnp.linspace(0, 8000, 123)
solver = make_solver(t_max=8000, t_points=t_points, rtol=1e-3, atol=1e-3)

In [ ]:
val0 = jnp.zeros(cells.n_cells)
#val0 = val0.at[slice(10,20)].set(10.0)

state = Species(
    tracer=val0,
)


In [ ]:
solution = solver(state, system)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(cells.centers[:], solution.ys.tracer.T[:,0::10]);

In [ ]:
%matplotlib widget
from matplotlib import animation


fig, ax = plt.subplots()

artists = []
for data in solution.ys.tracer:
    container = ax.plot(cells.centers, data, color="C0")
    artists.append(container)


ani = animation.ArtistAnimation(fig=fig, artists=artists, interval=40)
plt.show()